In [ ]:
import pandas as pd
import numpy as np

# 절대 모멘텀
1. 주식 데이터 로드 
2. 파생변수 년월 파생변수 생성 -> Date 컬럼에서 년-월 추출하여 대입
3. 년, 월 별 마지막날의 month_last_dt 데이터프레임을 생성
4. 전월의 종가를 가지는 파생변수 하나 생성
5. 전년도의 종가를 가지는 파생변수 하나 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익율 계산

In [ ]:
df = pd.read_csv("../csv/GDX.csv", index_col='Date')
df.head(3)

In [ ]:
# index를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
from datetime import datetime

In [ ]:
# datetime 라이브러리를 이용하여 시계열로 변경
test_df = df.copy()

In [ ]:
# case1
test_df = test_df.reset_index()
test_df['Date'] = test_df['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
test_df.set_index('Date')

In [ ]:
# case2
test_df.index

def change(x):
    return datetime.strptime(x, '%Y-%m-%d')

test_df.index = list(map(change, test_df.index))

In [ ]:
test_df.head(2)

In [ ]:
# map 함수
x = [1,2,3,4]
def change(x):
    return x**2

list(map(lambda y : y**2, x))

In [ ]:
df.index

In [ ]:
# 새로운 파생변수 년-월의 데이터를 대입
df['STD-YM'] = list(map(lambda x : datetime.strftime(x, '%Y-%m'), df.index))

In [ ]:
df.head(2)

In [ ]:
(df['STD-YM'] != df.shift(-1)['STD-YM']).iloc[24:30]

In [ ]:
# 월별 마지막 인덱스만 추출하여 새로운 데이터프레임에 대입

# case1 (shift 이용)
# 현재의 STD-YM의 값과 다음 행의 STD-YM의 값이 다른 경우 
df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]

In [ ]:
last_df = df.loc[df['STD-YM'] == "2006-06"]
last_df.tail(1)

In [ ]:
# case2 (for문)
month_list_df = pd.DataFrame()

_list = df['STD-YM'].unique()

for i in _list:
    last_df = df.loc[df['STD-YM'] == i].tail(1)
    month_list_df = pd.concat([month_list_df, last_df])

month_list_df

In [ ]:
x = pd.Series([1,1,1,2,2,4,4,4,4,7,7,7,7,7,7])

x.unique()

In [ ]:
# 전월의 종가(Adj Close)를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전년도의 종가(Adj Close)를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체

month_list_df['BF_1M'] = month_list_df.shift(1)['Adj Close'].fillna(0)

In [ ]:
month_list_df.head(3)

In [ ]:
month_list_df['BF_12M'] = month_list_df.shift(12)['Adj Close'].fillna(0)

In [69]:
month_list_df.iloc[[0,12], [4, 8]]

,Adj Close,BF_12M
Date,,
2006-05-31,35.771877,0.000000
2007-05-31,36.162109,35.771877


In [71]:
# 거래내역 추가 
# 구매 조건 -> ((전월 종가 / 전 년도 종가) - 1)이 값이 0보다 크고 무한대가 아닌 경우

df['trade'] = ""

for i in month_list_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_list_df.loc[i,'BF_1M'] / \
        month_list_df.loc[i,'BF_12M'] - 1

    # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
    flag = True if((momentum_index > 0) and (momentum_index != -np.inf ) \
        and (momentum_index != np.inf)) else False
    
    if flag:
        signal = 'buy'
    
    print('날짜 :', i, "모멘텀 인덱스 :", momentum_index, "flag :", flag, \
        "signal :", signal)
    
    df.loc[i, 'trade'] = signal

날짜 : 2006-05-31 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 2006-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-09-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-10-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-12-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-01-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-02-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-03-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-05-31 00:00:00 모멘텀 인덱스 : 0.02123525695897932 flag : True signal : buy
날짜 : 2007-06-29 00:00:00 모멘텀 인덱스 : 0.014827231670955676 flag : True signal : buy
날짜 : 2007-07-31 00:00:00 모멘텀 인덱스 : -0.019604388455949806 flag : False signal : 
날짜 : 2007-08-31

C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2280660222.py:10: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_list_df.loc[i,'BF_1M'] / \
C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2280660222.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_list_df.loc[i,'BF_1M'] / \


In [72]:
df['trade'].value_counts()

       3234
buy      61
Name: trade, dtype: int64

### 수익율 계산
1. 파생변수(return) 값은 1로 생성
2. rtn = 1.0, buy, sell 변수는 0으로 생성
3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 ""는 경우 구매 날
4. 전 행의 trade가 buy이고 현재의 trade가 ""인 경우 판매 날
5. 수익율 계산해서 return 대입

### 누적 수익율 계산
1. acc_rtn = 1.0 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입
3. 최종적으로 acc_rtn을 출력

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3295 entries, 2006-05-22 to 2019-06-24
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3295 non-null   float64
 1   High       3295 non-null   float64
 2   Low        3295 non-null   float64
 3   Close      3295 non-null   float64
 4   Adj Close  3295 non-null   float64
 5   Volume     3295 non-null   int64  
 6   STD-YM     3295 non-null   object 
 7   trade      3295 non-null   object 
dtypes: float64(5), int64(1), object(2)
memory usage: 360.7+ KB


In [76]:
len(df.index.unique())

3295

In [78]:
df.index.value_counts().sort_values(ascending=False)

2006-05-22    1
2006-06-09    1
2006-05-24    1
2006-05-25    1
2006-05-26    1
             ..
2019-06-17    1
2019-06-18    1
2019-06-19    1
2019-06-20    1
2019-06-24    1
Name: Date, Length: 3295, dtype: int64

In [ ]:
df['return'] = 1
rtn = 1.0
buy = 0
sell = 0

for i in df.index:
    # 구매한 날을 체크 , 구매가 지정
    if (df.loc[i, 'trade'] == "buy") and (df.shift(1).loc[i, 'trade'] == ""):
        buy = df.loc[i, 'Adj Close']
        print('진입일 :', i, '구입 가격 :', buy)
    # 판매한 날을 체크, 판매가 지정, 수익율 계산
    elif (df.loc[i, 'trade'] == "") and (df.shift(1).loc[i, 'trade'] == 'buy'):
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print('판매일 :', i, "판매 가격 :", sell, "수익율 :", rtn)



In [81]:
acc_rtn = 1.0

for i in df.index:
    acc_rtn *= df.loc[i, 'return']
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

누적 수익율 : 1.165001849964831


In [82]:
# 누적곱를 해주는 함수 
df['return'].cumprod()

Date
2006-05-22    1.000000
2006-05-23    1.000000
2006-05-24    1.000000
2006-05-25    1.000000
2006-05-26    1.000000
                ...   
2019-06-18    1.165002
2019-06-19    1.165002
2019-06-20    1.165002
2019-06-21    1.165002
2019-06-24    1.165002
Name: return, Length: 3295, dtype: float64